In [2]:
import os

In [3]:
os.chdir("../")
%pwd

'/Users/badaradiallo/Desktop/Kidney-Disease-Mlflow-DVC'

In [4]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Bls17/Kidney-Disease-Mlflow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Bls17"
os.environ["MLFLOW_TRACKING_PASSWORD"]="61087f41394d9ae01fd32f4ba34d8d0bd3cd9f48"

In [5]:
import tensorflow as tf 

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [8]:
from src.cnnClassifier.constant import *
from src.cnnClassifier.utils.commons import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.keras",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/Bls17/Kidney-Disease-Mlflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [ ]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


In [35]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [36]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
    raise e

[2025-07-10 14:31:58,701: INFO: commons: yaml file: config/config.yaml loaded successfully]
[2025-07-10 14:31:58,728: INFO: commons: yaml file: params.yaml loaded successfully]
[2025-07-10 14:31:58,750: INFO: commons: created directory at: artifacts]
Found 139 images belonging to 2 classes.


/Users/badaradiallo/Desktop/Kidney-Disease-Mlflow-DVC/env/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.2985 - loss: 3.7689
[2025-07-10 14:32:12,003: INFO: commons: json file saved at: scores.json]
[2025-07-10 14:32:12,004: INFO: commons: json file saved at: scores.json]
